In [0]:
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from torch.autograd import Variable
from PIL import Image
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch
import argparse
import os
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import numpy as np
import math
from glob import glob
from google.colab import drive

In [0]:
noOfEpochs = 20
input_image = (3, 200, 180)
eeta = 0.001
b1 = 0.5
interval = 100
batch_size = 32
b2 = 0.999
ldim = 200

os.makedirs("images", exist_ok=True)

In [0]:
class autoEncoder(nn.Module):
    def __init__(self):
        super(autoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(3*200*180,2000 ),
            nn.Tanh(),
            nn.Linear(2000, 1000),
            nn.Tanh(),
            nn.Linear(1000, 200),
            nn.Tanh(),
            nn.Linear(200, 100),   # compress to 3 features which can be visualized in plt
        )
        self.decoder = nn.Sequential(
            nn.Linear(100, 200),
            nn.Tanh(),
            nn.Linear(200, 1000),
            nn.Tanh(),
            nn.Linear(1000, 2000),
            nn.Tanh(),
            nn.Linear(2000, 3*200*180),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(input_image)), 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        legitity = self.model(img_flat)

        return legitity

In [4]:
drive.mount("/content/drive")
advLoss = torch.nn.BCELoss()
autoEncoder = autoEncoder()
discriminator = Discriminator()
autoEncoder_optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=eeta, betas=(b1, b2))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=eeta, betas=(b1, b2))
Tensor = torch.FloatTensor
dir_path = 'drive/My Drive/faces94'
files = glob(f"{dir_path}/**/**/*.jpg")
c2i = {
    "female": 0,
    "male":1,
    "malestaff": 2 
}
tloader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

total_dataset = torch.stack([tloader(Image.open(x)) for x in files])
total_labels = torch.tensor([c2i[x.split('/')[-3]] for x in files])

index_train, index_test = train_test_split(range(len(total_dataset)), test_size=0.2, random_state=102)

train_image = total_dataset[index_train]
train_label = total_labels[index_train]

test_image = total_dataset[index_test]
test_label = total_labels[index_test]

train_data = TensorDataset(train_image, train_label)
test_data = TensorDataset(test_image, test_label)

train_sample = RandomSampler(train_data)
test_sample = SequentialSampler(test_data)

train_loader = DataLoader(train_data, sampler=train_sample, batch_size=32)
test_loader = DataLoader(test_data, sampler=test_sample, batch_size=32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
for epoch in range(noOfEpochs):
    for i, (images,_) in enumerate(train_loader):
        legit = Variable(Tensor(images.size(0), 1).fill_(1.0), requires_grad=False)
        fake_autoEncoder_images = Variable(Tensor(images.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_images = Variable(images.type(Tensor))
        
        #print(real_images)

        autoEncoder_optimizer.zero_grad()
        
        # Sample noise as autoEncoder input
        z = Variable(Tensor(np.random.normal(0, 1, (32,108000))))

        # Generate a batch of images
        autoEncoder_images = autoEncoder(z)
        print(type(autoEncoder_images))
        # Loss measures generator's ability to fool the discriminator
        autoEncoder_loss = advLoss(discriminator(autoEncoder_images), legit)

        autoEncoder_loss.backward()
        autoEncoder_optimizer.step()

        discriminator_optimizer.zero_grad()

        # Measure discriminator's ability to classify real from autoEncoder samples
        real_loss = advLoss(discriminator(real_images), legit)
        autoEncoder_images_loss = advLoss(discriminator(autoEncoder_images.detach()), fake_autoEncoder_images)
        discriminator_loss = (real_loss + autoEncoder_images_loss) / 2

        discriminator_loss.backward()
        discriminator_optimizer.step()

        batches_done = epoch * len(train_loader) + i
        if batches_done % interval == 0:
            save_image(autoEncoder_images.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
            
        print("[Epoch = %d/%d] [Batch = %d/%d] [Discriminator loss = %f] [Generator loss = %f]" % (epoch, noOfEpochs, i, len(train_loader), discriminator_loss.item(), autoEncoder_loss.item()))
